In [ ]:
import os
from pathlib import Path
from transformers import AutoTokenizer, GPT2Model, AutoModelForCausalLM, AutoTokenizer, TextStreamer, BitsAndBytesConfig
import torch
device = 'cuda' # 'cpu'
torch.set_default_device(device)
os.environ['HF_HOME'] = '~/workspace/local_data/huggingface_cache/'

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_id = 'mistralai/Mistral-7B-Instruct-v0.2'
model_id = 'google/gemma-2b-it'
model_id = 'apple/OpenELM-3B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_id, 
                                             trust_remote_code=True,)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             device_map="auto", 
                                             torch_dtype=torch.bfloat16, 
                                             trust_remote_code=True,
                                            #  quantization_config=quantization_config
                                             )

In [ ]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")
streamer = TextStreamer(tokenizer, skip_prompt=True)
with torch.no_grad():
# Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**input_ids, streamer=streamer, max_new_tokens=100)